In [ ]:
import pandas as pd
data = pd.read_csv('./samples/')

## Comparing distributions

In [5]:
import pandas as pd
import numpy as np

# readin datasets (complete dataset and locate to imputed folder)
dataset = 'sim_1_tiny'
method = 'cart'
mr = 0.3
size = 5000
sample_id = 0
impute_num = 10
complete_data_path = './samples/' + dataset + '/complete_' + str(mr) + '_' + str(size) + '/sample_' + str(sample_id) + '.csv'
imputed_data_folder = './results/' + dataset + '/MCAR_' + str(mr) + '_' + str(size) + '/' + method + '/'

data = pd.read_csv(complete_data_path, header=None)
data.head()

,0,1,2,3
0,1.0,1.0,2.0,3.0
1,0.0,0.0,2.0,2.0
2,0.0,0.0,1.0,1.0
3,1.0,0.0,2.0,2.0
4,2.0,1.0,0.0,2.0


In [14]:
# divide cat/num type
numeric_variable_nums = dict([('boston', 12), ('house',8),('sim_1', 0),('sim_2',0), ('sim_1_tiny',0), ('sim_2_tiny',0)])
num_index = list(range(data.shape[1] - numeric_variable_nums[dataset], data.shape[1]))
cat_index = list(range(0, data.shape[1] - numeric_variable_nums[dataset]))
data.iloc[:, cat_index].head()

,0,1,2,3
0,1.0,1.0,2.0,3.0
1,0.0,0.0,2.0,2.0
2,0.0,0.0,1.0,1.0
3,1.0,0.0,2.0,2.0
4,2.0,1.0,0.0,2.0


In [18]:
# get all possible levels for categorical variable
all_levels = [np.unique(x) for x in data.iloc[:, cat_index].T]
all_levels_dict = dict(zip(data.columns[cat_index], all_levels))
all_levels_dict[cat_index[-1]]

array([3])